In [ ]:
!pip install einops
!pip install sentence_transformers

In [ ]:
pip install gdown

In [ ]:
!gdown --fuzzy https://drive.google.com/file/d/1aMTDpcJy5s6iRGQ1LLx9QYUuwOfLUIzv/view?usp=drive_link

In [ ]:
!unzip /kaggle/working/jin-AI.zip

In [ ]:
import numpy as np
import pandas as pd
import torch 
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer
import gc
from tqdm import tqdm
import os

In [ ]:
import pandas as pd

# Đọc file CSV
df = pd.read_csv('/kaggle/input/corpus/corpus_chunked.csv')
df.dropna(inplace=True)
half_index = len(df) // 2
df_half = df.iloc[:half_index]
df_half.to_csv('/kaggle/working/first_half.csv', index=False)
df_half = df_half[df_half['text'].str.split().str.len() >= 1]
last_cid = df_half['cid'].iloc[-1]
print("CID cuối cùng trong nửa đầu:", last_cid)

In [ ]:
len(df_half)

In [ ]:
len(df)

In [ ]:
import pandas as pd

# Đọc file CSV
df = pd.read_csv('/kaggle/input/corpus/corpus_chunked.csv')
df.dropna(inplace=True)
half_index = len(df) // 2
df_half = df.iloc[half_index:]
df_half.to_csv('/kaggle/working/second_half.csv', index=False)
df_half = df_half[df_half['text'].str.split().str.len() >= 1]
last_cid = df_half['cid'].iloc[-1]
print("CID cuối cùng trong nửa đầu:", last_cid)

In [ ]:
len(df_half)

In [ ]:
len(df)

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import h5py
import torch
from sentence_transformers import SentenceTransformer
class DocumentEncoder:
    def __init__(self, model_path="/kaggle/working/jin-AI/checkpoint-38440",
                 max_seq_length=1024, dtype=None, load_in_4bit=True, batch_size=8):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.dense_model = SentenceTransformer("/kaggle/working/jin-AI/checkpoint-38440", trust_remote_code=True)
        self.dense_model = self.dense_model.to(self.device)
        self.embedding_dim = 1024 
        self.dense_model.eval()
        self.task = 'text-matching'
        #self.task = "retrieval.passage"
        self.batch_size = batch_size

    def encode_document(self, text):
        """Mã hóa danh sách văn bản thành embeddings."""
        try:
            with torch.no_grad():
                embeddings = self.dense_model.encode(
                    text,
                    device=self.device,
                    show_progress_bar=False,
                    task=self.task,
                    max_length=1024
                )
            return embeddings
        except Exception as e:
            print(f"Error during encoding: {e}")
            return None

    def encode_and_save_remaining(self, csv_path, final_path):
        """Mã hóa dữ liệu từ file CSV và lưu embeddings vào file HDF5."""
        corpus_df = pd.read_csv(csv_path, dtype={'cid': str})  # Đảm bảo cột 'cid' là chuỗi
        with h5py.File(final_path, 'a') as f:
            # Tạo hoặc load datasets trong file HDF5
            if 'embeddings' not in f:
                dset_embeddings = f.create_dataset(
                    'embeddings',
                    shape=(0, self.embedding_dim),
                    maxshape=(None, self.embedding_dim),
                    dtype='float32'
                )
            else:
                dset_embeddings = f['embeddings']
    
            if 'cids' not in f:
                string_dtype = h5py.string_dtype(encoding='utf-8')  # Định nghĩa chuỗi Unicode
                dset_cids = f.create_dataset(
                    'cids',
                    shape=(0,),
                    maxshape=(None,),
                    dtype=string_dtype
                )
            else:
                dset_cids = f['cids']
    
            # Encode và lưu embeddings
            for start in tqdm(range(0, len(corpus_df), self.batch_size), desc="Encoding Batches"):
                end = min(start + self.batch_size, len(corpus_df))
                texts = corpus_df['text'][start:end].tolist()
                cids = corpus_df['cid'][start:end].tolist()
    
                # Kiểm tra `cids` hợp lệ
                if not all(cids):  # Kiểm tra giá trị rỗng
                    print(f"Warning: Empty CIDs detected in batch starting at index {start}. Skipping...")
                    continue

                try:
                    # Mã hóa văn bản
                    batch_embeddings = self.encode_document(texts)

                    if batch_embeddings is not None:
                        # Append embeddings vào dataset
                        current_size = dset_embeddings.shape[0]
                        dset_embeddings.resize(current_size + len(batch_embeddings), axis=0)
                        dset_embeddings[current_size:] = batch_embeddings

                        # Chuyển đổi CIDs sang chuỗi Unicode
                        cids_array = np.array(cids, dtype=h5py.string_dtype(encoding='utf-8'))

                        # Append cids vào dataset
                        dset_cids.resize(current_size + len(cids_array), axis=0)
                        dset_cids[current_size:] = cids_array
                except Exception as e:
                    print(f"Error processing batch {start}-{end}: {e}")
    
        print("Encoding and saving completed!")

In [ ]:
encoder = DocumentEncoder(batch_size=1)

# Encode the first half of the corpus and save to a single file
encoder.encode_and_save_remaining(csv_path = "/kaggle/working/first_half.csv", final_path = "first_half_dvt_embeddings.h5")

In [ ]:
encoder = DocumentEncoder(batch_size=1)

# Encode the first half of the corpus and save to a single file
encoder.encode_and_save_remaining(csv_path = "/kaggle/working/second_half.csv", final_path = "second_half_dvt_embeddings.h5")